In [1]:
import copy
from pprint import pprint

In [2]:
'''
## Objects
==========
Node = { <router_id> : {
            'num_ports'.         : _val_,
            'queue_arrival_rate' : [<input_port>][<output_port>],   # λ_(i→j)
            'queue_service_rate' : [<input_port>][<output_port>],   # μ_(i→j)
            'queue_occupancy'    : [<input_port>][<output_port>],   # q_(i→j) 
            <port_id> : {
                'agg_input_arrival_rate'  : _val_,   # λ_i
                'agg_input_service_rate'  : _val_,   # μ_i
                'agg_output_arrival_rate' : _val_,   # λ_j
                'agg_output_service_rate' : _val_,   # μ_i
        
                'queue_capacity'          : _val_,   # Q_i
        
                <output_port_id>          : { 'flow_ids': [...]},      # list of flows this path through the router
                }
            }
    }

Routing Table = {
    'current_node_id' : {
        'destination_node_id : [<current_output_port>, <next_input_port>, <next_node>],
        }
    }

Flow Table = {
    <flow_id> : {
        'rate'    : _val_,   # current flow rate
        'routers' : []       # routers along path




# Usage examples
================
> Sets the arrival rate for traffic entering input port i=1 and leaving output port j=2
Node['queue_arrival_rate'][1][2] = 10

> Sets the aggregate arrival rate of traffic leaving output port j=2
Node[2]['agg_output_arrival_rate'] = 30

'''


"\n## Objects\n==========\nNode = { <router_id> : {\n            'num_ports'.         : _val_,\n            'queue_arrival_rate' : [<input_port>][<output_port>],   # λ_(i→j)\n            'queue_service_rate' : [<input_port>][<output_port>],   # μ_(i→j)\n            'queue_occupancy'    : [<input_port>][<output_port>],   # q_(i→j) \n            <port_id> : {\n                'agg_input_arrival_rate'  : _val_,   # λ_i\n                'agg_input_service_rate'  : _val_,   # μ_i\n                'agg_output_arrival_rate' : _val_,   # λ_j\n                'agg_output_service_rate' : _val_,   # μ_i\n        \n                'queue_capacity'          : _val_,   # Q_i\n        \n                <output_port_id>          : { 'flow_ids': [...]},      # list of flows this path through the router\n                }\n            }\n    }\n\nRouting Table = {\n    'current_node_id' : {\n        'destination_node_id : [<current_output_port>, <next_input_port>, <next_node>],\n        }\n    }\n\nFlow

In [3]:
def SteadyRouter(rid):
    global routers

    # LOOP OVER ALL INPUT AND OUTPUT PORTS
    
    # Calculate λj for each output port
    for j in range(routers[rid]['num_ports']):
        routers[rid][j]['agg_output_arrival_rate'] = 0
        for i in range(routers[rid]['num_ports']):
            routers[rid][j]['agg_output_arrival_rate'] += routers[rid]['queue_arrival_rate'][i][j]
    
    # Calculate μi→j proportionally
    for i in range(routers[rid]['num_ports']):
        for j in range(routers[rid]['num_ports']):
            routers[rid]['queue_arrival_rate_old'][i][j] = routers[rid]['queue_arrival_rate'][i][j]
            
            if routers[rid][j]['agg_output_arrival_rate'] > 0:
                routers[rid]['queue_service_rate'][i][j] = \
                    (routers[rid]['queue_arrival_rate'][i][j] / routers[rid][j]['agg_output_arrival_rate']) *\
                        routers[rid][j]['agg_output_service_rate']
            else:
                routers[rid]['queue_service_rate'][i][j] = 0

    # Update queue occupancies; balance arrivals and service rates
    for i in range(routers[rid]['num_ports']):
        print(f"{routers[rid][i]['queue_capacity']}")
        qx = routers[rid][i]['queue_capacity'] # Remaining space in queue to be shared proportionally
        td = 0                           # Total rate differentials for proportional sharing

        for j in range(routers[rid]['num_ports']):
            # Group 1: same arrival and service rate
            if routers[rid]['queue_arrival_rate'][i][j] == routers[rid]['queue_service_rate'][i][j]:
                qx = qx - routers[rid]['queue_occupancy'][i][j]

            # Group 2: arrival rate less than service rate
            elif routers[rid]['queue_arrival_rate'][i][j] < routers[rid]['queue_service_rate'][i][j]:
                routers[rid]['queue_occupancy'][i][j] = 0
                routers[rid]['queue_service_rate'][i][j] = routers[rid]['queue_arrival_rate'][i][j]

            # Group 3: arrival rate more than service rate (now only accumulate rate differentials)
            else:
                td = td + (routers[rid]['queue_arrival_rate'][i][j] - routers[rid]['queue_service_rate'][i][j])

        # Group 3: arrival rate more than service rate
        for j in range(routers[rid]['num_ports']):
            if routers[rid]['queue_arrival_rate'][i][j] > routers[rid]['queue_service_rate'][i][j]:
                my_td = routers[rid]['queue_arrival_rate'][i][j] - routers[rid]['queue_service_rate'][i][j]
                routers[rid]['queue_occupancy'][i][j] = (my_td/td) * qx
                routers[rid]['queue_arrival_rate'][i][j] = routers[rid]['queue_service_rate'][i][j]

    # Update aggregate input rates
    for i in range(routers[rid]['num_ports']):
        routers[rid][i]['agg_input_arrival_rate'] = 0
        routers[rid][i]['agg_input_service_rate'] = 0
        for j in range(routers[rid]['num_ports']):
            routers[rid][i]['agg_input_arrival_rate'] += routers[rid]['queue_arrival_rate'][i][j]
            routers[rid][i]['agg_input_service_rate'] += routers[rid]['queue_service_rate'][i][j]

   # Update aggregate output service rates. Aggregate output arrival was calculate at function start
    for j in range(routers[rid]['num_ports']):
        routers[rid][j]['agg_output_arrival_rate'] = 0
        routers[rid][j]['agg_output_service_rate'] = 0
        for i in range(routers[rid]['num_ports']):
            routers[rid][j]['agg_output_arrival_rate'] += routers[rid]['queue_arrival_rate'][i][j]
            routers[rid][j]['agg_output_service_rate'] += routers[rid]['queue_service_rate'][i][j]

In [4]:
# Update the arrival rate for all cross-flows through the router
def UpdateArrival(rid):
    global routers
    
    for i in range(routers[rid]['num_ports']):
        for j in range(routers[rid]['num_ports']):
            routers[rid]['queue_arrival_rate'][i][j] = 0
            for flow_id in routers[rid][i][j]['flow_ids']:
                routers[rid]['queue_arrival_rate'][i][j] += flow_table[flow_id]['rate']

        routers[rid][i]['agg_input_arrival_rate'] = 0
        for j in range(routers[rid]['num_ports']):
            routers[rid][i]['agg_input_arrival_rate'] += routers[rid]['queue_arrival_rate'][i][j]

In [5]:
def SteadyNetwork():
    global routers
    global flow_table
    global occupancy

    routers_to_process = [] # list of routers to be steadied
    
    for rid in routers:
        UpdateArrival(rid)
        
        # Initialize input queuing occupancy
        if occupancy and rid in occupancy:
            for i in range(routers[rid]['num_ports']):
                sum_of_i = 0
                # sum over all j in occupancy[rid] that match this i
                for j in range(routers[rid]['num_ports']):
                    # .get() returns 0 if (i, j) not in occupancy[rid]
                    sum_of_i += occupancy[rid].get((i, j), 0)

                # Compare sum_of_i to queue_capacity at that port i
                capacity_i = routers[rid][i]['queue_capacity']                
                if sum_of_i > capacity_i:
                    print(f"WARNING: Router {rid}, input port {i} occupancy {sum_of_i} "
                          f"exceeds capacity {capacity_i}!")
                    
            for (i_j_tuple), initial_occ in occupancy[rid].items():
                (i, j) = i_j_tuple 
                routers[rid]['queue_occupancy'][i][j] = initial_occ
        else:        
            for i in range(routers[rid]['num_ports']):
                for j in range(routers[rid]['num_ports']):
                    if routers[rid][i]['agg_input_arrival_rate'] > 0:
                        routers[rid]['queue_occupancy'][i][j] = \
                            (routers[rid]['queue_arrival_rate'][i][j] / routers[rid][i]['agg_input_arrival_rate']) * \
                                routers[rid]['queue_occupancy'][i][j]
                    else:
                        routers[rid]['queue_occupancy'][i][j] = 0

        for j in range(routers[rid]['num_ports']):
            output_port_arrival_rate = 0
            for i in range(routers[rid]['num_ports']):
                output_port_arrival_rate += routers[rid]['queue_arrival_rate'][i][j]
                routers[rid][j]['agg_output_service_rate'] = min(output_port_arrival_rate, link_transmission_rate)

        for j in range(routers[rid]['num_ports']):
            if routers[rid][j]['agg_output_service_rate'] > 0:
                # Add the router if there’s flow through it
                routers_to_process.append(rid)
                break
                
    # Repeat until all routers reach equilibrium
    for rid in routers_to_process:
        SteadyRouter(rid)

        for i in range(routers[rid]['num_ports']):
            for j in range(routers[rid]['num_ports']):
                if routers[rid]['queue_arrival_rate'][i][j] != routers[rid]['queue_arrival_rate_old'][i][j]:

                    #if routers[rid]['queue_arrival_rate_old'][i][j] > 0:
                    rate_multiplier =  routers[rid]['queue_arrival_rate'][i][j]/routers[rid]['queue_arrival_rate_old'][i][j]

                    for flow_id in routers[rid][i][j]['flow_ids']:
                        # Update rate of all crossing flows
                        flow_table[flow_id]['rate'] = rate_multiplier * flow_table[flow_id]['rate']

                        # Reconsider routers along the flow path
                        for flow_router in flow_table[flow_id]['routers']:
                            if flow_router != rid:
                                UpdateArrival(flow_router)
                                if flow_router not in routers_to_process:
                                    routers_to_process.append(flow_router)

                    routers[rid]['queue_arrival_rate_old'][i][j] = routers[rid]['queue_arrival_rate'][i][j]

In [ ]:
### This section contains the inputs to the model and variables that must be initilized before steadying the network
link_transmission_rate = 10
router_queue_capacity = 10
router_num_ports = 3
rids = [2, 3]

'''
node0                                     node5
 ┌─┐                                      ┌─┐  
 └─┘                                      └─┘  
  │      ┌──────────┐      ┌──────────┐    ▲   
  └─────►│0         │      │         2├────┘   
         │  node2  2├─────►│0  node3  │        
  ┌─────►│1         │      │         1├────┐   
  │      └──────────┘      └──────────┘    ▼   
 ┌─┐                                      ┌─┐  
 └─┘                                      └─┘  
node1                                     node4
'''

# Initialize table of flows and their properties 
flow_table = {0: {'source': 0, 'destination': 5, 'demand': 20, 'rate': 10, 'routers': []},
              1: {'source': 1, 'destination': 4, 'demand': 20, 'rate': 10, 'routers': []}}


# Static routing table for finding path through the network
routing_table = {0:          # nodeid
                     {5: (0, 0, 2)}, # dst : (curr_output_port, next_input_port, next_node)
                 1:
                     {4: (0, 1, 2)},
                 2:
                     {4: (2, 0, 3),
                      5: (2, 0, 3)},
                 3:
                     {4: (1, 0, 4),
                      5: (2, 0, 5)},
                }

# this is Q_i
occupancy = {
    2: {            # router: {(input port, output port): occupancy, }
        (0, 2): 3,  # q_(i→j)
        (0, 1): 5,
        (1, 2): 4,
        (1, 0): 3,
        (1, 1): 1,
        (2, 0): 2},  # Router 2, (i=0→j=2) is 5, (i=1→j=2) is 8
    3: { 
        (0, 1): 7,
        (0, 2): 2, }
}

# Define base router
baserouter = {'num_ports' : router_num_ports,
          'queue_arrival_rate': [[]] * router_num_ports,   # λ_(i→j)'
          'queue_service_rate': [[]] * router_num_ports,   # μ_(i→j)
          'queue_occupancy'   : [[]] * router_num_ports,   # q_(i→j)
          'queue_arrival_rate_old': [[]] * router_num_ports, 
         }
for i in range(baserouter['num_ports']): # each input port
    baserouter['queue_arrival_rate'][i] = [0] * router_num_ports
    baserouter['queue_service_rate'][i] = [0] * router_num_ports
    baserouter['queue_occupancy'][i] = [0] * router_num_ports
    baserouter['queue_arrival_rate_old'][i] = [0] * router_num_ports
    
    baserouter[i] = {} 
    baserouter[i]['agg_input_arrival_rate']  = 0   # λ_i
    baserouter[i]['agg_input_service_rate']  = 0   # μ_i
    baserouter[i]['agg_output_arrival_rate'] = 0   # λ_j
    baserouter[i]['agg_output_service_rate'] = 0   # μ_i
    baserouter[i]['queue_capacity'] = router_queue_capacity   # Q_i
    for j in range(baserouter['num_ports']):
        baserouter[i][j] = {'flow_ids': []} # list of flows this path through the router


# Initialize all router nodes
routers = {}
for rid in rids:
    routers[rid] = copy.deepcopy(baserouter)

# Add flows to routers and routers to flow table
for flow_id in flow_table.keys():
    # Read flow information
    rate = flow_table[flow_id]['rate']
    src = flow_table[flow_id]['source']
    dst = flow_table[flow_id]['destination']

    # Walk through the nodes along the flow's routed path
    (curr_output_port, next_input_port, next_node) = routing_table[src][dst]
    while next_node != dst:
        next_output_port = routing_table[next_node][dst][0]
        
        # Add flow to router input port arrivals
        routers[next_node]['queue_arrival_rate'][next_input_port][next_output_port] = rate
        routers[next_node]['queue_service_rate'][next_input_port][next_output_port] = rate
        routers[next_node][next_input_port][next_output_port]['flow_ids'].append(flow_id)

        # Add router to flow table
        flow_table[flow_id]['routers'].append(next_node)
        
        # Move to next hop
        (curr_output_port, next_input_port, next_node) = routing_table[next_node][dst]
    

In [7]:
# Please run the previous to initialize the network before running this cell

SteadyNetwork()

print("\n=== Steadying Network complete===========================================")
print("\n=== Routers ===")
pprint(routers)
print("\n=== Flow table: ===")
pprint(flow_table)


10
10
10
10
10
10

=== Steadying Network complete===========================================

=== Routers ===
{2: {0: {0: {'flow_ids': []},
         1: {'flow_ids': []},
         2: {'flow_ids': [0]},
         'agg_input_arrival_rate': 5.0,
         'agg_input_service_rate': 5.0,
         'agg_output_arrival_rate': 0,
         'agg_output_service_rate': 0,
         'queue_capacity': 10},
     1: {0: {'flow_ids': []},
         1: {'flow_ids': []},
         2: {'flow_ids': [1]},
         'agg_input_arrival_rate': 5.0,
         'agg_input_service_rate': 5.0,
         'agg_output_arrival_rate': 0,
         'agg_output_service_rate': 0,
         'queue_capacity': 10},
     2: {0: {'flow_ids': []},
         1: {'flow_ids': []},
         2: {'flow_ids': []},
         'agg_input_arrival_rate': 0,
         'agg_input_service_rate': 0.0,
         'agg_output_arrival_rate': 10.0,
         'agg_output_service_rate': 10.0,
         'queue_capacity': 10},
     'num_ports': 3,
     'queue_arrival_rat